# F1 Strategy - EDA

**Goal:** build a clean race-driver table for strategy modeling.
**Scope:** race sessions only.
**Inputs:** `sessions_all.csv`, `sessions_result_all.csv`, `starting_grid_all.csv`, `pit_all.csv`, `race_control_all.csv`, `weather_all.csv`
**Outputs:** `data/processed/race_driver_master.csv` ready for visualization and feature engineering.

**Keys**
- Primary key: `session_key` + `driver_number`.
- Race-level joins: `session_key`.

**Principles**
- Minimal code. Few assumptions. No heavy imputations in EDA.
- Report missingness. Do not over-fix.

In [1]:
import pandas as pd
from pathlib import Path

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
DATA = Path('../data/openf1_full')
OUT  = Path('../data/processed')
OUT.mkdir(parents = True, exist_ok = True)

## Data loading

We load only the tables that are needed for strategy:
- Sessions and results for the base race outcomes.
- Starting grid and pits for start positions and service info.
- Race control and weather for race-level context.

In [4]:
sessions = pd.read_csv(DATA / 'sessions_all.csv')
results  = pd.read_csv(DATA / 'session_result_all.csv')
sessions.shape, results.shape

((323, 14), (1836, 11))

In [5]:
grid = pd.read_csv(DATA / 'starting_grid_all.csv')
pits = pd.read_csv(DATA / 'pit_all.csv')
grid.shape, pits.shape

((298, 5), (3033, 6))

In [6]:
racectl = pd.read_csv(DATA / 'race_control_all.csv')
weather = pd.read_csv(DATA / 'weather_all.csv')
racectl.shape, weather.shape

((6901, 10), (12179, 10))

## Restrict to race sessions

We keep only **Race** sessions.
This ensures that downstream metrics (position, pits, control flags, weather) refer to the same race scope.

In [7]:
is_race = sessions.get('session_type', pd.Series("", index = sessions.index)).eq('Race')
if "session_name" in sessions.columns:
    is_race |= sessions['session_name'].str.contains('Race', case = False, na = False)

race_sessions = (
    sessions.loc[is_race, ['session_key', 'meeting_key', 'year']]
        .drop_duplicates()
        .reset_index(drop = True)
)
race_sessions.shape

(77, 3)

In [8]:
race_sessions['year'].value_counts().sort_index()

year
2023    28
2024    30
2025    19
Name: count, dtype: int64

## Base results

Create the base table from `session_result_all` limited to race `session_key`.
Keep onlly columns that are meaningful for strategy and are present in the data version we have.

In [9]:
race_results = results[results['session_key'].isin(race_sessions['session_key'])].copy()
race_results = race_results.merge(race_sessions, on = 'session_key', how = 'left')

In [10]:
if 'meeting_key_x' in race_results.columns and 'meeting_key_y' in race_results.columns:
    race_results['meeting_key'] = race_results['meeting_key_x'].fillna(race_results['meeting_key_y'])
    race_results = race_results.drop(columns = ['meeting_key_x', 'meeting_key_y'])

In [11]:
if 'number_of_laps' in race_results.columns:
    race_results = race_results.rename(columns = {'number_of_laps': 'laps_completed'})

In [12]:
keep = [
    'session_key', 'meeting_key', 'year', 'driver_number',
    'position', 'points', 'laps_completed',
    'dnf', 'dns', 'dsq', 'gap_to_leader', 'duration'
]
race_results = race_results[[c for c in keep if c in race_results.columns]].reset_index(drop = True)
race_results.head(3)

,session_key,meeting_key,year,driver_number,position,points,laps_completed,dnf,dns,dsq,gap_to_leader,duration
0,7953.0,1141,2023,1,1.0,25.0,57.0,False,False,False,0,5636.736
1,7953.0,1141,2023,11,2.0,18.0,57.0,False,False,False,11.987,5648.723
2,7953.0,1141,2023,14,3.0,15.0,57.0,False,False,False,38.637,5675.373


## Pit-stop features

Aggregate per `(session_key, driver_number)`:
- `pit_stop_count` - number of pit entries.
- `first_pit_lap`  - earliest lap with pit.
- `avg_pit_duration` - mean service time.

Missing value policy (EDA level):
- If as driver has a race result and no pit rows, set `pit_stop_count = 0`.
- Leave `first_pit_lap` and `avg_pit_duration` as NaN when `pit_stop_count = 0`.

In [13]:
pits_slim = pits[['session_key', 'driver_number', 'lap_number', 'pit_duration']]

In [14]:
pits_agg = (
    pits_slim.groupby(['session_key', 'driver_number'])
            .agg(pit_stop_count   = ('pit_duration', 'size'),
                 first_pit_lap    = ('lap_number', 'min'),
                 avg_pit_duration = ('pit_duration', 'mean'))
            .reset_index()
)

In [15]:
pits_agg['pit_stop_count'] = pits_agg['pit_stop_count'].astype('Int64')
pits_agg.head()

,session_key,driver_number,pit_stop_count,first_pit_lap,avg_pit_duration
0,9102,1,2,26.0,22.25
1,9102,2,2,17.0,23.15
2,9102,4,3,1.0,27.50
3,9102,10,2,19.0,23.45
4,9102,11,2,27.0,21.60


## Starting grid

Keep a single `grid_position` per `(session_key, driver_number)`.
Ensure numeric type for downstream arithmetic.

In [16]:
grid_slim = (
    grid[['session_key', 'driver_number', 'position']]
)

In [17]:
grid_slim = (
    grid_slim.sort_values(['session_key', 'driver_number', 'position'])
                .groupby(['session_key', 'driver_number'], as_index = False)
                .first()
)
grid_slim['position'] = pd.to_numeric(grid_slim['position'], errors = 'coerce').astype('Int64')
grid_slim.head()

,session_key,driver_number,position
0,9278,1,3
1,9278,4,10
2,9278,10,17
3,9278,11,2
4,9278,14,8


In [18]:
race_sessions['session_key'] = pd.to_numeric(race_sessions['session_key'], errors = 'coerce')
racectl['session_key']       = pd.to_numeric(racectl['session_key'], errors = 'coerce')

rc = racectl[racectl['session_key'].isin(race_sessions['session_key'])].copy()

In [19]:
text_cols = rc.select_dtypes(include = 'object').columns.tolist()
if text_cols:
    rc['_all_text'] = rc[text_cols].fillna('').astype(str).agg(''.join, axis = 1).str.upper()
else:
    rc['_all_text'] = ''

## Race control flags (SC/VSC)

Heuristic detection across all text columns:
- SC patterns: `SC`, `Safety Car`. `Full Course Yellow`.
- VSC patterns: `VSC`, `Virtual Safety Car`.

Aggregate to race level:
- `had_sc`, `had_vsc`, and `had_sc_or_vsc`.

In [20]:
pat_sc  = r"(?:\bSC\b|SAFETY\s*CAR|SAFETYCAR|SC\s*DEPLOYED|SAFETY\s*CAR\s*DEPLOYED|FULL\s*COURSE\s*YELLOW|FCY)"
pat_vsc = r"(?:\bVSC\b|VIRTUAL\s*SAFETY\s*CAR|VIRTUALSAFETYCAR|VSC\s*DEPLOYED|VIRTUAL\s*SC)"

rc['is_sc']  = rc['_all_text'].str.contains(pat_sc, regex = True, na = False)
rc['is_vsc'] = rc['_all_text'].str.contains(pat_vsc, regex = True, na = False)

In [21]:
sc_flags = (
    rc.groupby('session_key', as_index = False)
        .agg(had_sc = ('is_sc', 'any'), had_vsc = ('is_vsc', 'any'))
)
sc_flags['had_sc_or_vsc'] = sc_flags['had_sc'] | sc_flags['had_vsc']
sc_flags.head()

,session_key,had_sc,had_vsc,had_sc_or_vsc
0,7779,True,False,True
1,7787,True,True,True
2,7953,True,True,True
3,9069,True,True,True
4,9070,True,False,True


## Weather per race

Compute race-level aggregates:
- `mean_air_temp`, `mean_track_temp` (means).
- `rain_any` (any non-zero rainfall).

These are race-leve covariates for later visualization and modeling.

In [22]:
wea = weather[weather['session_key'].isin(race_sessions['session_key'])].copy()
wea.shape

(11176, 10)

In [23]:
air   = 'air_temperature'
track = 'track_temperature'
rain  = 'rainfall'

In [24]:
for c in [air, track, rain]:
    if c:
        wea[c] = pd.to_numeric(wea[c], errors = 'coerce')
agg = {}
if air:   agg[air]     = 'mean'
if track: agg[track]   = 'mean'
if rain:  agg[rain]    = 'max'

weather_agg = wea.groupby('session_key').agg(agg).reset_index()

rename_map = {}
if air:   rename_map[air]   = 'mean_air_temp'
if track: rename_map[track] = 'mean_track_temp'
if rain:  rename_map[rain]  = 'rain_any'

weather_agg = weather_agg.rename(columns = rename_map)
if 'rain_any' in weather_agg:
    weather_agg['rain_any'] = weather_agg['rain_any'].fillna(0).gt(0)

weather_agg.head()

,session_key,mean_air_temp,mean_track_temp,rain_any
0,7779,26.091892,31.792568,False
1,7787,17.449550,30.139640,False
2,7953,27.431677,31.011801,False
3,9069,22.201220,36.831707,True
4,9070,24.860625,41.210000,False


In [25]:
def norm_keys(df):
    df = df.copy()
    df["session_key"] = pd.to_numeric(df["session_key"], errors = "coerce").astype("Int64")
    if "driver_number" in df.columns:
        df["driver_number"] = pd.to_numeric(df["driver_number"], errors = "coerce").astype("Int64")
    return df

race_results = norm_keys(race_results)
grid_slim    = norm_keys(grid_slim)
pits_agg     = norm_keys(pits_agg)
sc_flags     = norm_keys(sc_flags)
weather_agg  = norm_keys(weather_agg)
results      = norm_keys(results)

## Build the master table

Join order and keys:
1. Base `race_results`.
2. `grid_slim` on `(session_key, driver_number)`.
3. `pits_agg` on `(session_key, driver_number)`.
4. `sc_flags` on `session_key`.
5. `weather_agg` on `session_key`.

Normalization:
- Cast `session_key` and `driver_number` to numeric nullable typers.
- Drop or rename colliding columns (e.g., `position_x`, `position_y`).

In [26]:
master = race_results.copy()
master.shape

(1536, 12)

In [27]:
master = master.merge(grid_slim, on = ["session_key","driver_number"], how = "left")
master.shape

(1536, 13)

In [28]:
master = master.merge(pits_agg, on = ["session_key","driver_number"], how = "left")
master.shape

(1536, 16)

In [29]:
master = master.merge(sc_flags, on = "session_key", how = "left")
master = master.merge(weather_agg, on = "session_key", how = "left")
master.head(3)

,session_key,meeting_key,year,driver_number,position_x,points,laps_completed,dnf,dns,dsq,gap_to_leader,duration,position_y,pit_stop_count,first_pit_lap,avg_pit_duration,had_sc,had_vsc,had_sc_or_vsc,mean_air_temp,mean_track_temp,rain_any
0,7953,1141,2023,1,1.0,25.0,57.0,False,False,False,0,5636.736,<NA>,<NA>,NaN,NaN,True,True,True,27.431677,31.011801,False
1,7953,1141,2023,11,2.0,18.0,57.0,False,False,False,11.987,5648.723,<NA>,<NA>,NaN,NaN,True,True,True,27.431677,31.011801,False
2,7953,1141,2023,14,3.0,15.0,57.0,False,False,False,38.637,5675.373,<NA>,<NA>,NaN,NaN,True,True,True,27.431677,31.011801,False


In [30]:
if "position_x" in master.columns:
    master = master.rename(columns = {"position_x": "position"})
if "position_y" in master.columns:
    master = master.drop(columns = ["position_y"])

In [31]:
laps_col = "laps_completed" if "laps_completed" in master.columns else ("number_of_laps" if "number_of_laps" in master.columns else None)
if laps_col:
    raced = master[laps_col].notna()
    master.loc[raced & master["pit_stop_count"].isna(), "pit_stop_count"] = 0
    master["pit_stop_count"] = master["pit_stop_count"].astype("Int64")

## Status flags

Extract DNS and DSQ from concatenated text in `results`.
Optional heuristic for DNF:
- `position` is NaN and laps > 0.

Keep booleans as nullable types. Do not over-interpret in EDA.

In [32]:
res_text = results.select_dtypes(include = "object").fillna("").astype(str).agg(" ".join, axis = 1).str.upper()
res_status = results[["session_key","driver_number"]].copy()
res_status["status_text"] = res_text

pat_dns = r"\bDNS\b|DID NOT START|WITHDRAWN|WITHDREW"
pat_dsq = r"\bDSQ\b|DISQUALIFIED|EXCLUDED"

res_status["dns_alt"] = res_status["status_text"].str.contains(pat_dns, regex = True, na = False)
res_status["dsq_alt"] = res_status["status_text"].str.contains(pat_dsq, regex = True, na = False)

master = master.merge(res_status[["session_key", "driver_number", "dns_alt", "dsq_alt"]],
                      on = ["session_key", "driver_number"], how = "left")

master["dns"] = master.get("dns", False) | master["dns_alt"].fillna(False)
master["dsq"] = master.get("dsq", False) | master["dsq_alt"].fillna(False)
master.drop(columns = ["dns_alt", "dsq_alt"], inplace = True, errors = "ignore")

In [33]:
master.shape, master[["session_key","driver_number"]].duplicated().sum()

((1536, 21), np.int64(0))

In [34]:
master

,session_key,meeting_key,year,driver_number,position,points,laps_completed,dnf,dns,dsq,gap_to_leader,duration,pit_stop_count,first_pit_lap,avg_pit_duration,had_sc,had_vsc,had_sc_or_vsc,mean_air_temp,mean_track_temp,rain_any
0,7953,1141,2023,1,1.0,25.0,57.0,False,False,False,0,5636.736,0,NaN,NaN,True,True,True,27.431677,31.011801,False
1,7953,1141,2023,11,2.0,18.0,57.0,False,False,False,11.987,5648.723,0,NaN,NaN,True,True,True,27.431677,31.011801,False
2,7953,1141,2023,14,3.0,15.0,57.0,False,False,False,38.637,5675.373,0,NaN,NaN,True,True,True,27.431677,31.011801,False
3,7953,1141,2023,55,4.0,12.0,57.0,False,False,False,48.052,5684.788,0,NaN,NaN,True,True,True,27.431677,31.011801,False
4,7953,1141,2023,44,5.0,10.0,57.0,False,False,False,50.977,5687.713,0,NaN,NaN,True,True,True,27.431677,31.011801,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,9912,1268,2025,10,16.0,0.0,52.0,False,False,False,+1 LAP,NaN,1,49.0,25.0,False,False,False,26.761111,43.288194,False
1532,9912,1268,2025,43,17.0,0.0,52.0,False,False,False,+1 LAP,NaN,1,33.0,24.9,False,False,False,26.761111,43.288194,False
1533,9912,1268,2025,18,18.0,0.0,52.0,False,False,False,+1 LAP,NaN,1,49.0,38.4,False,False,False,26.761111,43.288194,False
1534,9912,1268,2025,14,NaN,0.0,24.0,True,False,False,NaN,NaN,1,20.0,25.1,False,False,False,26.761111,43.288194,False


## Derived features and types

Type normalization:
- Integers: ids, positions, counts.
- Floats:   times, temperature.
- Booleans: control flags, status flags.

Derived:
- `gain_on_start = grid_position - position`.
- `had_pit = pit_stop_count > 0`.

In [35]:
int_cols = [c for c in ["year", "meeting_key", "session_key", "driver_number",
                        "grid_position", "position", "pit_stop_count", "first_pit_lap", "laps_completed"]
            if c in master.columns]
for c in int_cols:
    master[c] = pd.to_numeric(master[c], errors="coerce").astype("Int64")

float_cols = [c for c in ["points", "avg_pit_duration", "mean_air_temp", "mean_track_temp",
                          "gap_to_leader", "duration"]
              if c in master.columns]
for c in float_cols:
    master[c] = pd.to_numeric(master[c], errors="coerce")

In [36]:
bool_cols = [c for c in ["had_sc", "had_vsc", "had_sc_or_vsc", "rain_any", "dnf", "dns", "dsq"] if c in master.columns]
for c in bool_cols:
    master[c] = master[c].astype("boolean")

In [37]:
if {"grid_position","position"}.issubset(master.columns):
    master["gain_on_start"] = (master["grid_position"] - master["position"]).astype("Int64")

if "pit_stop_count" in master.columns:
    master["had_pit"] = master["pit_stop_count"].fillna(0).gt(0).astype("boolean")

In [38]:
master.dtypes

session_key           Int64
meeting_key           Int64
year                  Int64
driver_number         Int64
position              Int64
points              float64
laps_completed        Int64
dnf                 boolean
dns                 boolean
dsq                 boolean
gap_to_leader       float64
duration            float64
pit_stop_count        Int64
first_pit_lap         Int64
avg_pit_duration    float64
had_sc              boolean
had_vsc             boolean
had_sc_or_vsc       boolean
mean_air_temp       float64
mean_track_temp     float64
rain_any            boolean
had_pit             boolean
dtype: object

## Quality checks

- Missingness report for core columns.
- Uniqueness of `(session_key, driver_number)`.
- Sanity check: correlation between `grid_position` and `position` (Spearman).

In [39]:
cols = [c for c in ["grid_position", "position", "points", "laps_completed",
                    "pit_stop_count", "first_pit_lap", "avg_pit_duration", "gain_on_start"]
        if c in master.columns]
missing = (master[cols].isna().mean().mul(100).round(1).sort_values(ascending = False)).to_frame("missing_%")
missing

dup = master[["session_key", "driver_number"]].duplicated().sum()
print("duplicates (session_key, driver_number):", dup)

if {"grid_position", "position"}.issubset(master.columns):
    print(master[["grid_position", "position"]].dropna().corr(method = "spearman"))

duplicates (session_key, driver_number): 0


## Export

Save the master table for visualization and modeling:
- `data/processed/race_driver_master.csv`
- Optionally `parquet` if available.

In [40]:
OUT.mkdir(parents = True, exist_ok = True)
master.to_csv(OUT / "race_driver_master.csv", index = False)

master.shape

(1536, 22)